# Get average mkv for each stock in each fund

In [51]:
import tushare as ts
import pandas as pd
import time
from tqdm import tqdm
pro = ts.pro_api('5d2dd2c56bb822ac0e818aaa4b0b344f95c7d5e7b9c83c69e2ed90ff')

In [52]:
raw_data = pd.read_csv("data/raw_portfolio_list_whole.csv", index_col=0)

C:\Users\wangs\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [53]:
raw_data

,ts_code,ann_date,end_date,symbol,mkv
0,512980.SH,20201028,20200930,600637.SH,7.318376e+07
1,512980.SH,20201028,20200930,002027.SZ,3.056744e+08
2,512980.SH,20201028,20200930,002131.SZ,7.179756e+07
3,512980.SH,20201028,20200930,002555.SZ,1.854357e+08
4,512980.SH,20201028,20200930,002602.SZ,1.048872e+08
...,...,...,...,...,...
1593286,040001.OF,20180122,20171231,601336.SH,5.616000e+07
1593287,040001.OF,20180122,20171231,601699.SH,5.429853e+07
1593288,040001.OF,20180122,20171231,300316.SZ,4.749706e+07
1593289,040001.OF,20180122,20171231,600048.SH,4.733600e+07


In [54]:
fund_list = raw_data.groupby('ts_code', as_index=False).first()['ts_code']

In [55]:
fund_list

0       000001.OF
1       000003.OF
2       000004.OF
3       000007.OF
4       000008.OF
          ...    
3977    740101.OF
3978    750001.OF
3979    750005.OF
3980    762001.OF
3981    770001.OF
Name: ts_code, Length: 3982, dtype: object

In [56]:
ann_days = raw_data.groupby(['ts_code', 'ann_date'], as_index=False).first().groupby('ts_code').count().drop(columns=['end_date', 'symbol', 'mkv'])

In [57]:
ann_days

,ann_date
ts_code,
000001.OF,18
000003.OF,12
000004.OF,12
000007.OF,1
000008.OF,18
...,...
740101.OF,18
750001.OF,18
750005.OF,18


In [58]:
sum_fund_stock = raw_data.drop(columns=['ann_date', 'end_date']).groupby(['ts_code', 'symbol'], as_index=False).sum()

In [59]:
sum_fund_stock

,ts_code,symbol,mkv
0,000001.OF,000002.SZ,7.265244e+08
1,000001.OF,000009.SZ,4.671139e+06
2,000001.OF,000034.SZ,7.598948e+06
3,000001.OF,000063.SZ,7.101895e+07
4,000001.OF,000069.SZ,3.396000e+07
...,...,...,...
745719,770001.OF,603939.SH,1.656200e+06
745720,770001.OF,688081.SH,7.560280e+04
745721,770001.OF,688090.SH,1.223481e+05
745722,770001.OF,688181.SH,8.677254e+04


In [60]:
def ff(x):
    day_num = ann_days.loc[x['ts_code']]
    return x['mkv']/day_num
new_col = sum_fund_stock.apply(lambda x: ff(x), axis=1)
sum_fund_stock["avg_mkv"] = new_col

In [61]:
sum_fund_stock

,ts_code,symbol,mkv,avg_mkv
0,000001.OF,000002.SZ,7.265244e+08,4.036247e+07
1,000001.OF,000009.SZ,4.671139e+06,2.595077e+05
2,000001.OF,000034.SZ,7.598948e+06,4.221638e+05
3,000001.OF,000063.SZ,7.101895e+07,3.945497e+06
4,000001.OF,000069.SZ,3.396000e+07,1.886667e+06
...,...,...,...,...
745719,770001.OF,603939.SH,1.656200e+06,9.201111e+04
745720,770001.OF,688081.SH,7.560280e+04,4.200156e+03
745721,770001.OF,688090.SH,1.223481e+05,6.797117e+03
745722,770001.OF,688181.SH,8.677254e+04,4.820697e+03


In [62]:
sum_fund_stock = sum_fund_stock.drop(columns='mkv')

In [63]:
sum_fund_stock.to_csv("data/sum_fund_stock.csv")